In [11]:
import os
import sys
project_dir = os.path.join(os.pardir, os.pardir)
sys.path.append(project_dir)

import dotenv
dotenv_path = os.path.join(project_dir, '.env')
dotenv.load_dotenv(dotenv_path)

from IPython.display import display

import time
import numpy as np
import pandas as pd
import geopandas as gpd
import json
from sqlalchemy import MetaData, create_engine, extract, select
from sqlalchemy.engine.url import URL
from sqlalchemy.sql import or_
import datetime
from pytz import timezone
from shapely.geometry import Point
from geojson import Feature

import glob

from src.data.processing_func import (get_direction, connect_database, extract_geo_sections)
from src.data.load_func import (extract_jps,
                                transf_flow_features,
                                transf_flow_labels)

pd.options.display.max_columns = 30
pd.options.display.max_rows = 200

What the block below does is:
1. Connect to the "jams" table of the database. See database documentation for more information;
2. Retrieve 3 months of jams data as a Pandas dataframe;

SELECT JAMS
SELECT DATAFILES

In [2]:
#Connection and initial setup
DATABASE = {
'drivername': os.environ.get("db_drivername"),
'host': os.environ.get("db_host"), 
'port': os.environ.get("db_port"),
'username': os.environ.get("db_username"),
'password': os.environ.get("db_password"),
'database': os.environ.get("db_database"),
}

meta = connect_database(DATABASE)

start_point = datetime.date(day=1, month=10, year=2017)
periods = [(7,9), (17,19)]
num_weeks=14
#
#for i in range(0,num_weeks):
#    date_begin = start_point + datetime.timedelta(days=7*(i))
#    date_end = start_point + datetime.timedelta(days=7*(i+1))
#    temp_df_jps = extract_jps(meta, date_begin, date_end, periods=periods, weekends=False, summary=False, limit=None)
#    print("Batch " + str(i+1))
#    print(temp_df_jps.shape)
#    temp_df_jps.to_csv(project_dir + "/data/interim/temp_df_jps_" + \
#                       str(start_point.year) + \
#                       "_" + \
#                       str(start_point.month) + \
#                       "_" + \
#                       str(start_point.day) + \
#                       "_" + \
#                       "week" + str(i+1) + ".csv"
#                      )

DELETE THE BLOCK BELOW

In [3]:
#Get processed datasets
filenames = glob.glob(project_dir+"/data/interim/temp_df_jps_2017_10_1*")
filenames.sort()
frame = pd.DataFrame()
list_ = []
for file_ in filenames:
    df = pd.read_csv(file_, index_col=0)
    list_.append(df)
df_jps = pd.concat(list_)
display(df_jps.shape)
display(df_jps.sample(5))

(760635, 18)

,MgrcDateStart,JpsId,SctnId,JamId,JamIndLevelOfTraffic,JamQtdLengthMeters,JamSpdMetersPerSecond,JamTimeDelayInSeconds,JamDscCoordinatesLonLat,JamSpdKmPerHour,LonDirection,LatDirection,MajorDirection,date,hour,minute,period,minute_bin
42342,2017-10-19 07:41:00-02:00,6565313.0,5467.0,538528.0,3.0,1759.0,2.625000,511.0,"[{'y': -26.339918, 'x': -48.841444}, {'y': -26...",9.45,West,North,Norte/Sul,2017-10-19,7,41,-1,30 a 44
7468,2018-01-05 17:52:00-02:00,8093869.0,2033.0,870952.0,3.0,434.0,3.105556,72.0,"[{'x': -48.81534, 'y': -26.318369}, {'x': -48....",11.18,West,South,Norte/Sul,2018-01-05,17,52,1,45 a 59
26002,2017-12-06 07:47:00-02:00,7573086.0,15892.0,748658.0,3.0,2167.0,4.069444,348.0,"[{'x': -48.818894, 'y': -26.296046}, {'x': -48...",14.65,West,South,Leste/Oeste,2017-12-06,7,47,-1,45 a 59
58689,2017-12-08 17:52:00-02:00,7660150.0,8207.0,767394.0,3.0,1442.0,2.991667,329.0,"[{'x': -48.845717, 'y': -26.307351}, {'x': -48...",10.77,East,South,Norte/Sul,2017-12-08,17,52,1,45 a 59
9900,2017-10-09 18:06:00-03:00,6334258.0,8114.0,495374.0,3.0,2431.0,3.494444,426.0,"[{'x': -48.865925, 'y': -26.324797}, {'x': -48...",12.58,East,North,Leste/Oeste,2017-10-09,18,6,1,0 a 14


Cross-reference jams data with georeferenced sections from geo

In [ ]:
#LOAD SECTIONS SHAPEFILE OR WKT, CROSS WITH JAMS

In [5]:
# Filter holidays from list of holidays in Brazil, from ANBIMA
holidays = pd.read_excel(project_dir + "/data/external/feriados_nacionais.xls", skip_footer=9)
holidays["Data"] = holidays["Data"].dt.date
holiday_list = holidays["Data"].tolist()
df_jps = df_jps[~df_jps["date"].isin(holiday_list)]

#Since mere intersections of jams and sections count as jam for the entire section
#there is an unavoidable overcounting of jams, even taken into account the jam's and 
#section's directions. For simplicity sake, let's ignore that.

#Let's drop blocked congestions. We'll treat them as outliers.
df_jps = df_jps[df_jps["JamIndLevelOfTraffic"] != 5.0]
display(df_jps.shape)

#Aggregate traffic in slots of 15 minutes
jps_per_timeslot = df_jps.groupby(["SctnId", "hour",
                                 "minute_bin", "LonDirection","LatDirection"])\
                         .agg({"JpsId": ['count'],
                                "JamQtdLengthMeters": ["mean"],
                                "JamSpdKmPerHour": ["mean"],
                                "JamTimeDelayInSeconds": ["mean"],
                                "JamIndLevelOfTraffic": ["mean"],
                                "period": ["max"],
                                })
jps_per_timeslot.columns = [''.join(col_name).strip() for col_name in jps_per_timeslot.columns.values]
print(jps_per_timeslot.shape)
jps_per_timeslot.sample(5)


(757535, 18)

(50394, 6)


,,,,,JpsIdcount,JamQtdLengthMetersmean,JamSpdKmPerHourmean,JamTimeDelayInSecondsmean,JamIndLevelOfTrafficmean,periodmax
SctnId,hour,minute_bin,LonDirection,LatDirection,,,,,,
10867.0,17,15 a 29,East,North,12,208.000000,5.337500,126.000000,3.750000,1
8886.0,7,0 a 14,West,South,11,1168.454545,15.515455,141.363636,3.090909,-1
15961.0,18,0 a 14,East,North,1,1274.000000,31.240000,62.000000,1.000000,1
8868.0,8,0 a 14,West,South,14,1313.214286,14.225714,177.642857,2.857143,-1
7383.0,8,30 a 44,East,South,29,516.827586,7.841724,183.551724,3.310345,-1


In [6]:
#Merge dataset with total number of waze signals, for each bin
jps_per_timeslot.reset_index(level=["SctnId", "LonDirection","LatDirection"], inplace=True)

wazesignals_per_timeslot = df_jps.groupby(["hour", "minute_bin"]).agg({"MgrcDateStart": [pd.Series.nunique]})
wazesignals_per_timeslot.columns = ["".join(x) for x in wazesignals_per_timeslot.columns.ravel()]
jps_per_timeslot = jps_per_timeslot.join(wazesignals_per_timeslot, how="outer")

#For highly meshed networks it is possible that jams from neighbor streets be double counted in other sections.
#We are accepting that, but we need to cap the probability to 1 in case the numbers of jams is bigger than the
#numbers of Waze signals.
jps_per_timeslot["slot_traffic_prob"] = jps_per_timeslot["JpsIdcount"]/jps_per_timeslot["MgrcDateStartnunique"]
jps_per_timeslot["slot_traffic_prob"] = jps_per_timeslot["slot_traffic_prob"].clip(upper=1)
print(jps_per_timeslot.shape)
jps_per_timeslot.sample(5)

(50394, 11)


SctnId LonDirection LatDirection  JpsIdcount  \
hour minute_bin                                                  
17   30 a 44     10792.0         East        South           2   
     45 a 59     15394.0         East        South          32   
18   0 a 14       1181.0         West        South           2   
     0 a 14       3982.0         West        North           1   
     15 a 29      3802.0         West        South           4   

                 JamQtdLengthMetersmean  JamSpdKmPerHourmean  \
hour minute_bin                                                
17   30 a 44                 3390.00000            13.085000   
     45 a 59                 1154.15625             8.967188   
18   0 a 14                   385.00000             9.980000   
     0 a 14                   399.00000            11.850000   
     15 a 29                  678.50000            16.045000   

                 JamTimeDelayInSecondsmean  JamIndLevelOfTrafficmean  \
hour minute_bin                                                        
17   30 a 44                     808.50000                    4.0000   
     45 a 59                     391.46875                    3.4375   
18   0 a 14                       70.00000                    3.0000   
     0 a 14                       78.00000                    3.0000   
     15 a 29                      65.00000                    2.0000   

                 periodmax  MgrcDateStartnunique  slot_traffic_prob  
hour minute_bin                                                      
17   30 a 44             1                   210           0.009524  
     45 a 59             1                   210           0.152381  
18   0 a 14              1                   210           0.009524  
     0 a 14              1                   210           0.004762  
     15 a 29             1                   210           0.019048

In [7]:
jps_per_timeslot["notraffic_prob"] = 1 - jps_per_timeslot["slot_traffic_prob"]
jps_per_timeslot["weighted_JamSpdKmPerHourmean"] = jps_per_timeslot["slot_traffic_prob"]*jps_per_timeslot["JamSpdKmPerHourmean"]
jps_per_timeslot["weighted_JamQtdLengthMetersmean"] = jps_per_timeslot["slot_traffic_prob"]*jps_per_timeslot["JamQtdLengthMetersmean"]
jps_per_timeslot["weighted_JamTimeDelayInSecondsmean"] = jps_per_timeslot["slot_traffic_prob"]*jps_per_timeslot["JamTimeDelayInSecondsmean"]
jps_per_timeslot["weighted_JamIndLevelOfTrafficmean"] = jps_per_timeslot["slot_traffic_prob"]*jps_per_timeslot["JamIndLevelOfTrafficmean"]

traffic_indicators = jps_per_timeslot.groupby(["SctnId", "LonDirection", "LatDirection",
                                               "periodmax"]).agg({"notraffic_prob": np.prod,
                                                                  "slot_traffic_prob": np.sum,
                                                                  "weighted_JamSpdKmPerHourmean": np.sum,
                                                                  "weighted_JamQtdLengthMetersmean": np.sum,
                                                                  "weighted_JamTimeDelayInSecondsmean": np.sum,
                                                                  "weighted_JamIndLevelOfTrafficmean": np.sum}) 

traffic_indicators["traffic_prob"] = 1 - traffic_indicators["notraffic_prob"]
traffic_indicators["JamSpdKmPerHourmean"] = traffic_indicators["weighted_JamSpdKmPerHourmean"] / traffic_indicators["slot_traffic_prob"]
traffic_indicators["JamQtdLengthMetersmean"] = traffic_indicators["weighted_JamQtdLengthMetersmean"] / traffic_indicators["slot_traffic_prob"]
traffic_indicators["JamTimeDelayInSecondsmean"] = traffic_indicators["weighted_JamTimeDelayInSecondsmean"] / traffic_indicators["slot_traffic_prob"]
traffic_indicators["JamIndLevelOfTrafficmean"] = traffic_indicators["weighted_JamIndLevelOfTrafficmean"] / traffic_indicators["slot_traffic_prob"]
traffic_indicators.drop(["weighted_JamSpdKmPerHourmean",
                        "weighted_JamQtdLengthMetersmean",
                        "weighted_JamTimeDelayInSecondsmean",
                        "weighted_JamIndLevelOfTrafficmean",
                        "slot_traffic_prob"], axis=1, inplace=True)

print(traffic_indicators.shape)
traffic_indicators.sort_values("traffic_prob", ascending=False).head(5)

(12490, 6)


,,,,notraffic_prob,traffic_prob,JamSpdKmPerHourmean,JamQtdLengthMetersmean,JamTimeDelayInSecondsmean,JamIndLevelOfTrafficmean
SctnId,LonDirection,LatDirection,periodmax,,,,,,
2388.0,West,South,1,0.000003,0.999997,9.064874,1034.357492,320.897588,3.617777
8738.0,West,South,1,0.000004,0.999996,9.146053,1037.777390,319.764184,3.607898
14047.0,West,South,1,0.000004,0.999996,8.980130,1036.022165,323.725801,3.626795
2389.0,West,South,1,0.000006,0.999994,9.060348,1046.979216,325.267997,3.620174
8322.0,East,South,1,0.000012,0.999988,9.607132,563.421696,157.360347,3.196631


DELETE CODE BELOW. THIS HAS BEEN DONE ABOVE.

In [8]:
def get_main_direction(x):
    if x["StreetDirection"] == "Norte/Sul":
        return x["LatDirection"]
    elif x["StreetDirection"] == "Leste/Oeste":
        return x["LonDirection"]

#Merge dataset with official street sections from the Municipality
geo_sections = extract_geo_sections(meta, main_buffer=10)
traffic_indicators.reset_index(inplace=True)
geo_traffic_indicators = geo_sections.merge(traffic_indicators, how="inner", on="SctnId")
geo_traffic_indicators["Sentido"] = geo_traffic_indicators.apply(lambda x: get_main_direction(x), axis=1)

columns = {"SctnId": "Codigo do Trecho",
           "SctnCodRua": "Codigo da Rua",       
           "SctnDscNome": "Rua",
           "SctnIdArcgis": "CodArcgis",
           "SctnQtdMetrosAcumulados": "Metrica",
           "LonDirection": "Direcao L/O",
           "LatDirection": "Direcao N/S",
           "Sentido": "Sentido",
           "periodmax": "period",
           "traffic_prob":"Probabilidade de Transito",
           "JamSpdKmPerHourmean": "Velocidade Media (km/h)",
           "JamQtdLengthMetersmean": "Fila media (m)",
           "JamTimeDelayInSecondsmean": "Atraso medio (s)",
           "JamIndLevelOfTrafficmean": "Nivel medio de congestionamento (0 a 5)",
           "SctnDscCoordxUtmComeco": "SctnDscCoordxUtmComeco",
           "SctnDscCoordyUtmComeco": "SctnDscCoordyUtmComeco",
           "SctnDscCoordxUtmMeio": "SctnDscCoordxUtmMeio",
           "SctnDscCoordyUtmMeio": "SctnDscCoordyUtmMeio",
           "SctnDscCoordxUtmFinal": "SctnDscCoordxUtmFinal",
           "SctnDscCoordyUtmFinal": "SctnDscCoordyUtmFinal",
         }
geo_traffic_indicators.rename(columns=columns, inplace=True)
col_list = [col for col in columns.values()]
col_list.append("section_LineString")
geo_traffic_indicators = geo_traffic_indicators[col_list]

print(geo_traffic_indicators.shape)
geo_traffic_indicators.sort_values(["Probabilidade de Transito", "Atraso medio (s)"], ascending=False, inplace=True)
geo_traffic_indicators.head(10)

(12490, 21)


,Codigo do Trecho,Codigo da Rua,Rua,CodArcgis,Metrica,Direcao L/O,Direcao N/S,Sentido,period,Probabilidade de Transito,Velocidade Media (km/h),Fila media (m),Atraso medio (s),Nivel medio de congestionamento (0 a 5),SctnDscCoordxUtmComeco,SctnDscCoordyUtmComeco,SctnDscCoordxUtmMeio,SctnDscCoordyUtmMeio,SctnDscCoordxUtmFinal,SctnDscCoordyUtmFinal,section_LineString
1397,2388,6936,OTTOKAR DOERFFEL,18530,598,West,South,West,1,0.999997,9.064874,1034.357492,320.897588,3.617777,713569.9322,7.087803e+06,713602.977080,7.087822e+06,713639.3705,7.087835e+06,POLYGON ((-48.86027092304061 -26.3134803794554...
6858,8738,6936,OTTOKAR DOERFFEL,24999,708,West,South,West,1,0.999996,9.146053,1037.777390,319.764184,3.607898,713495.1530,7.087723e+06,713531.209538,7.087764e+06,713569.9322,7.087803e+06,POLYGON ((-48.86100415168816 -26.3140322095625...
10555,14047,6936,OTTOKAR DOERFFEL,30426,418,West,South,West,1,0.999996,8.980130,1036.022165,323.725801,3.626795,713739.4233,7.087837e+06,713765.212500,7.087853e+06,713791.0017,7.087868e+06,POLYGON ((-48.85839700701319 -26.3130386950682...
1403,2389,6936,OTTOKAR DOERFFEL,18531,521,West,South,West,1,0.999994,9.060348,1046.979216,325.267997,3.620174,713739.4233,7.087837e+06,713690.472874,7.087841e+06,713639.3705,7.087835e+06,POLYGON ((-48.85934181437936 -26.3134695470942...
6478,8322,3723,FLORIANOPOLIS,24568,340,East,South,East,1,0.999988,9.607132,563.421696,157.360347,3.196631,715740.0582,7.086806e+06,715773.460300,7.086752e+06,715806.8624,7.086699e+06,POLYGON ((-48.83822475403328 -26.3228380722825...
10483,13990,3723,FLORIANOPOLIS,30369,214,East,South,East,1,0.999986,9.602576,563.187761,157.488045,3.197565,715671.1895,7.086913e+06,715705.623850,7.086859e+06,715740.0582,7.086806e+06,POLYGON ((-48.83856901236984 -26.3223585139055...
7055,8839,9288,TUIUTI,25102,5590,East,South,South,1,0.999986,6.674238,723.201867,390.616256,3.529349,717735.4477,7.097672e+06,717822.564900,7.097472e+06,717909.6821,7.097272e+06,POLYGON ((-48.81860484266484 -26.2276036052062...
3445,5171,3723,FLORIANOPOLIS,21345,440,East,South,East,1,0.999985,9.617448,565.502648,157.754350,3.195007,715806.8624,7.086699e+06,715833.057800,7.086656e+06,715859.2532,7.086614e+06,POLYGON ((-48.83761163183942 -26.3236965472258...
6200,8110,6936,OTTOKAR DOERFFEL,24354,358,West,South,West,1,0.999982,8.881647,1057.248045,333.409669,3.643095,713791.0017,7.087868e+06,713825.219800,7.087897e+06,713859.4379,7.087927e+06,POLYGON ((-48.85807244633456 -26.3127786290922...
11754,15374,9288,TUIUTI,31769,5660,East,South,South,1,0.999981,6.691872,729.678026,394.006612,3.524488,717735.4477,7.097672e+06,717721.567600,7.097704e+06,717707.6875,7.097736e+06,POLYGON ((-48.82074156513088 -26.2238116116251...


In [ ]:
#Save as GeoJSON
file_name = "traffic_indicators_" + \
                       str(start_point.year) + \
                       "_" + \
                       str(start_point.month) + \
                       "_" + \
                       str(start_point.day) + \
                       "_" + str(num_weeks) + "_weeks.geoJSON"
                        
#geo_traffic_indicators.to_file(project_dir + "/data/interim/" + file_name, driver="GeoJSON")

#Save as CSV

def add_geojson(df):
    df["geojson"] = df.apply(lambda row: json.dumps(Feature(geometry=row.section_LineString)), axis=1)
    return df

geo_traffic_indicators = geo_traffic_indicators.pipe(add_geojson)

file_name = "traffic_indicators_" + \
                       str(start_point.year) + \
                       "_" + \
                       str(start_point.month) + \
                       "_" + \
                       str(start_point.day) + \
                       "_" + str(num_weeks) + "_weeks.csv"
                        
geo_traffic_indicators.to_csv(project_dir + "/data/interim/" + file_name)

## Etapa 1 - Rankeamento de Vias

### Cálculos realizados com dados referentes ao período de 01/20/2017 a 21/10/2018, das 07:00 às 09:00 e das 17:00 às 19:00.